<a href="https://colab.research.google.com/github/Alberta0/Purchase-Prediction2/blob/main/Next_Customer_Purchase_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R

# Load in the libraries

library(dplyr) # for data manipulation
library(tidyr) # for data manipulation
library(lubridate) # for date data type

In [ ]:
%%R
# importing in the dataset and selecting target variables
df <- read.csv('ecommerce_customer_data(1).csv') %>%
  select(Customer.ID, Purchase.Date)
df$Customer.ID <- as.character(df$Customer.ID)

# view data
head(df, n = 5)

  Customer.ID    Purchase.Date
1       46251 08/09/2020 09:38
2       46251 05/03/2022 12:56
3       46251 23/05/2022 18:18
4       46251 12/11/2020 13:13
5       13593 27/11/2020 17:55


In [ ]:
%%R
# pivoting the data for easy consumption, per unique customer
# A slice of the entire dataset was used because the program was stretched thin on rows > 10,000
pivot_df <- df %>%
  slice(1:10000) %>%
  mutate(row = row_number()) %>%
  pivot_wider(names_from = Customer.ID,
              values_from = Purchase.Date) %>%
  select(-row)

# creates a new dataframe to hold the transposed dataset
newPiv <- as.data.frame(t(pivot_df))


Created a function to move all values within the data to the left and NA's moved to the right for each row, this fucntion is used to handle NAs

In [ ]:
%%R
collapse_NAs <- function(row) {
  non_na_values <- row[!is.na(row)]
  na_values <- rep(NA, length(row) - length(non_na_values))
  return(c(non_na_values, na_values))
}

# Apply function to sort NAs
newPiv2 <- as.data.frame(t(apply(newPiv, 1, collapse_NAs)))

# Select only columns with values
newPiv2 <- newPiv2 %>%
  select(1:14)

# view data
head(newPiv2, n = 5)

                    V1               V2               V3               V4
46251 08/09/2020 09:38 05/03/2022 12:56 23/05/2022 18:18 12/11/2020 13:13
13593 27/11/2020 17:55 07/03/2023 14:17 15/04/2023 03:02 27/03/2021 21:23
28805 13/09/2023 04:24 31/03/2021 09:50 18/01/2021 22:42 07/01/2020 12:57
28961 25/04/2021 23:55 13/01/2020 09:57 18/06/2023 21:34 10/09/2021 00:39
12163 18/12/2021 17:49 20/06/2020 05:56 08/07/2023 13:36 01/07/2023 12:36
                    V5               V6               V7               V8
46251             <NA>             <NA>             <NA>             <NA>
13593 05/05/2020 20:14             <NA>             <NA>             <NA>
28805 12/02/2021 20:33 02/07/2020 02:54             <NA>             <NA>
28961 01/06/2023 19:07             <NA>             <NA>             <NA>
12163 16/11/2022 08:56 07/04/2020 04:24 28/11/2020 16:48 21/08/2020 09:23
                    V9  V10  V11  V12  V13  V14
46251             <NA> <NA> <NA> <NA> <NA> <NA>
13593           

In [ ]:
%%R
# Convert dates data in to date data types, then sort dates in sequential timeline
# Convert all columns to Date type
newPiv4 <- newPiv2 %>% mutate(across(everything(), ~ as.Date(., format = "%d/%m/%Y %H:%S")))

# Function to sort dates in each row and place them in the correct order
sort_dates_in_row <- function(row) {
  sorted_dates <- sort(na.omit(row))
  sorted_dates <- c(sorted_dates, rep(NA, length(row) - length(sorted_dates)))
  return(sorted_dates)
}

# Apply the function to each row of the data frame and save in new dataframe
df_sorted <- t(apply(newPiv4, 1, sort_dates_in_row))
df_sorted <- as.data.frame(df_sorted)

# view dataframe
head(df_sorted, n = 5)

              V1         V2         V3         V4         V5         V6
46251 2020-09-08 2020-11-12 2022-03-05 2022-05-23       <NA>       <NA>
13593 2020-05-05 2020-11-27 2021-03-27 2023-03-07 2023-04-15       <NA>
28805 2020-01-07 2020-07-02 2021-01-18 2021-02-12 2021-03-31 2023-09-13
28961 2020-01-13 2021-04-25 2021-09-10 2023-06-01 2023-06-18       <NA>
12163 2020-04-07 2020-06-20 2020-08-21 2020-11-28 2021-12-18 2022-11-16
              V7         V8         V9  V10  V11  V12  V13  V14
46251       <NA>       <NA>       <NA> <NA> <NA> <NA> <NA> <NA>
13593       <NA>       <NA>       <NA> <NA> <NA> <NA> <NA> <NA>
28805       <NA>       <NA>       <NA> <NA> <NA> <NA> <NA> <NA>
28961       <NA>       <NA>       <NA> <NA> <NA> <NA> <NA> <NA>
12163 2023-04-24 2023-07-01 2023-07-08 <NA> <NA> <NA> <NA> <NA>


In [ ]:
%%R
# Ensuring dates are in date data type across the dataframe
newPiv5 <- df_sorted %>% mutate(across(everything(), ~ as.Date(., format = "%Y-%m-%d")))

# view dataframe
head(newPiv5, n = 5)

              V1         V2         V3         V4         V5         V6
46251 2020-09-08 2020-11-12 2022-03-05 2022-05-23       <NA>       <NA>
13593 2020-05-05 2020-11-27 2021-03-27 2023-03-07 2023-04-15       <NA>
28805 2020-01-07 2020-07-02 2021-01-18 2021-02-12 2021-03-31 2023-09-13
28961 2020-01-13 2021-04-25 2021-09-10 2023-06-01 2023-06-18       <NA>
12163 2020-04-07 2020-06-20 2020-08-21 2020-11-28 2021-12-18 2022-11-16
              V7         V8         V9  V10  V11  V12  V13  V14
46251       <NA>       <NA>       <NA> <NA> <NA> <NA> <NA> <NA>
13593       <NA>       <NA>       <NA> <NA> <NA> <NA> <NA> <NA>
28805       <NA>       <NA>       <NA> <NA> <NA> <NA> <NA> <NA>
28961       <NA>       <NA>       <NA> <NA> <NA> <NA> <NA> <NA>
12163 2023-04-24 2023-07-01 2023-07-08 <NA> <NA> <NA> <NA> <NA>


The next step will calculate the difference in dates for each row, per customer, to create differences in number of days and sort them into rows.

In [ ]:
%%R
compute_diffs <- function(row) {
  dates <- as.Date(row, origin = "1970-01-01")
  diffs <- diff(dates)
  return(diffs)
}

# Apply the function row-wise and create a new data frame
df_diffs <- t(apply(newPiv5 %>% select(V1:V14), 1, compute_diffs))
df_diffs <- as.data.frame(df_diffs)

# Include column to serve as unique identifier
df_diffs <- df_diffs %>%
  mutate(ID = rownames(df_diffs)) %>%
  select(ID, everything())

# Keeping dataframe in days
df_diffs$ID <- as.numeric(df_diffs$ID)

# view dataframe
head(df_diffs, n = 5)

         ID  V2  V3  V4  V5  V6  V7 V8 V9 V10 V11 V12 V13 V14
46251 46251  65 478  79  NA  NA  NA NA NA  NA  NA  NA  NA  NA
13593 13593 206 120 710  39  NA  NA NA NA  NA  NA  NA  NA  NA
28805 28805 177 200  25  47 896  NA NA NA  NA  NA  NA  NA  NA
28961 28961 468 138 629  17  NA  NA NA NA  NA  NA  NA  NA  NA
12163 12163  74  62  99 385 333 159 68  7  NA  NA  NA  NA  NA


Created a function filters through the dataframe, selecting the customer ID, then calculates the mean/average for all the days between the customer's purchase. Using Moving Average, it predicts the next number of days to the next purchase

In [ ]:
%%R

NextPurchase <- function(IDFilter) {
  # filter customer ID
  filteredID <- df_diffs %>% filter(ID == IDFilter) %>%
    # remove NAs
    select(where(~ !any(is.na(.))))
  # Stating target in number of days
  AssumedNext <- 120
  # calculating average
  average <- mean(as.numeric(filteredID[,2:ncol(filteredID)]), na.rm = T)
  # identifying last purchase day
  lastValue <- filteredID[, ncol(filteredID)]
  # using moving average that is adding average to last day to predict next purchase in days
  pred <- round((average + lastValue), digits = 0)
  # probability of next purchase using the calculated next purchase and target days
  prob <- round(((AssumedNext/pred) * 100), digits = 2)
  prob <- ifelse(prob > 100, 100, prob)
  print(paste('Customer ID -', IDFilter, 'next purchase is approximately in', pred, 'days'))
  print(paste0('Customer ID - ', IDFilter, ' has a ', prob,'%', ' chance of buying in ', AssumedNext, ' days'))
}

In [ ]:
%%R
# using desired Customer ID
NextPurchase(25638)

[1] "Customer ID - 25638 next purchase is approximately in 264 days"
[1] "Customer ID - 25638 has a 45.45% chance of buying in 120 days"
